In [3]:
import sagemaker
import uuid

sagemaker_session = sagemaker.Session()
print('SageMaker version: ' + sagemaker.__version__)

SageMaker version: 2.72.1


In [4]:
bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-cnn-mnist'

role = sagemaker.get_execution_role()

In [5]:
role

'arn:aws:iam::113935906496:role/service-role/AmazonSageMaker-ExecutionRole-20220129T162909'

In [6]:
bucket

'sagemaker-us-east-1-113935906496'

In [7]:
pip install torchvision==0.5.0 --no-cache-dir

     |████████████████████████████████| 4.0 MB 9.8 MB/s            
     |████████████████████████████████| 753.4 MB 301 kB/s             10
  Attempting uninstall: torch
    Found existing installation: torch 1.7.1
    Uninstalling torch-1.7.1:
      Successfully uninstalled torch-1.7.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.8.2
    Uninstalling torchvision-0.8.2:
      Successfully uninstalled torchvision-0.8.2
Note: you may need to restart the kernel to use updated packages.


In [8]:
import torch,torchvision
torchvision.__version__,torch.__version__

('0.5.0', '1.4.0')

In [9]:
from torchvision import datasets, transforms

In [10]:
datasets.MNIST(
    "data",
    download = True,
    transform = transforms.Compose(
        [transforms.ToTensor(),transforms.Normalize((0.1378,),(0.3081,))]
    )
)

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1378,), std=(0.3081,))
           )

In [11]:
inputs = sagemaker_session.upload_data(path="data",bucket=bucket,key_prefix=prefix)
print("input spec (in this case just an s3 path): {}".format(inputs))

input spec (in this case just an s3 path): s3://sagemaker-us-east-1-113935906496/sagemaker/DEMO-pytorch-cnn-mnist


In [14]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="mnist.py",
    role = role,
    framework_version = "1.4.0",
    py_version = "py3",
    instance_count = 1,
    instance_type= "ml.c4.xlarge",
    hyperparameters = {
        "epochs" : 2, "backend" : "gloo"
    }
)

In [15]:
estimator.fit({"training" : inputs})

2022-02-02 04:05:51 Starting - Starting the training job...
2022-02-02 04:06:14 Starting - Launching requested ML instancesProfilerReport-1643774751: InProgress
......
2022-02-02 04:07:15 Starting - Preparing the instances for training............
2022-02-02 04:09:15 Downloading - Downloading input data......
2022-02-02 04:10:16 Training - Downloading the training image.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-02-02 04:10:30,627 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-02-02 04:10:30,629 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-02-02 04:10:30,641 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-02-02 04:10:30,872 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-02-02 04:10:31,270 sagemaker-containers INFO     Module default_user_module_name

Test set: Average loss: 0.2056, Accuracy: 9400/10000 (94%)
Train Epoch: 2 [6400/60000 (11%)] Loss: 0.739589
Train Epoch: 2 [12800/60000 (21%)] Loss: 0.400858
Train Epoch: 2 [19200/60000 (32%)] Loss: 0.394676
Train Epoch: 2 [25600/60000 (43%)] Loss: 0.648297
Train Epoch: 2 [32000/60000 (53%)] Loss: 0.594082
Train Epoch: 2 [38400/60000 (64%)] Loss: 0.431972
Train Epoch: 2 [44800/60000 (75%)] Loss: 0.284116
Train Epoch: 2 [51200/60000 (85%)] Loss: 0.540223
Train Epoch: 2 [57600/60000 (96%)] Loss: 0.299974
Test set: Average loss: 0.1334, Accuracy: 9590/10000 (96%)
Saving the model.
2022-02-02 04:11:27,927 sagemaker-training-toolkit INFO     Reporting training SUCCESS

2022-02-02 04:11:56 Uploading - Uploading generated training model
2022-02-02 04:11:56 Completed - Training job completed
ProfilerReport-1643774751: NoIssuesFound
Training seconds: 164
Billable seconds: 164


In [16]:
estimator.model_data

's3://sagemaker-us-east-1-113935906496/pytorch-training-2022-02-02-04-05-50-859/output/model.tar.gz'

In [17]:
predictor = estimator.deploy(initial_instance_count=1,instance_type="ml.t2.medium")

-------------!

In [19]:
# import gzip
# import numpy as np
# import os
# import random

# data_dir = "data/MNIST/raw"
# with gzip.open(os.path.join(data_dir, "t10k-images-idx3-ubyte.gz"),'rb') as f:
#     images = np.frombuffer(f.read(),np.int8, offset=16).reshape(-1,28,28).astype(np.float32)
    
# mask = random.sample(range(len(images)),16)
# mask = np.array(mask,dtype=np.int)
# data = images[mask]

In [28]:
# response = predictor.predict(np.expand_dims(data,axis =2))
# print("Raw prediction result:")
# print(response)
# print()
# labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[0])

In [23]:
from IPython.display import HTML
HTML(open("input.html").read())

In [27]:
import numpy as np
image = np.array([data],dtype=np.float32)
response = predictor.predict(image)
prediction = response.argmax(axis=1)[0]
prediction

5